# PyMem3DG Tutorial 6 - Extended Applications 2: protein dynamics
`Cuncheng Zhu, Christopher T. Lee`

This tutorial covers how `Mem3DG` models protein dynamics. In previous tutorials, we have been focusing on the mechanics of membrane in response to current configuration and material properties, while the other side of the coin is the chemical response: how protein density $\phi$ respond to the condition? 

Just as the mechanical force that drives the shape evolution, chemical potential drives the protein evolution. They are the shape variation and chemical variation of the free energy, respectively. The tutorial aims to provide the fundamental and practical API maniputation. Please refer to the documentation and publication for details of the theory. 

To demonstrate, this tutorial will model the protein evolution on a realistic dendritic spine model processed using GAMer(https://github.com/ctlee/gamer). Instead of evolving the shape of the geometry, we evolve the protein distribution on a given geometry. The dynamics of protein will be governed by curvature-sensing property of membrane-bound protein, in-plane diffusion of membrane protein, as well as out-of-plane protein-membrane adsorption. 

In [5]:
import pymem3dg as dg 

spine = "sample_meshes/boundarySpine.ply"
spineFace, spineVertex = dg.processSoup(spine)

For a untreated mesh soup, it might need to be processed before solving using `Mem3DG`. For this purpose, built-in function `processSoup` helps by internally reconstructing the mesh, merging identical vertices and stripping faces with duplicated vertices. 

Since we are only interested in protein dynamics, we need to disable shape variation and enable protein variation. 

The initialization of protein density `protein0` should be between 0 and 1 to avoid evoking the interior penalty method used to contain the range of $\phi$ during the simulation. 

The only boundary condition for protein density that current `Mem3DG` supports is the `"pin"` boundary condition, which keep the boundary constant as `protein0`.  

In [6]:
o = dg.Options()
p = dg.Parameters()

o.isShapeVariation = False
o.isProteinVariation = True
o.proteinBoundaryCondition = "pin"

p.protein0 = [0.1]

# bending
p.Kb = 8.22e-5
p.H0c = 10
# Dirichlet
p.eta = 0.01
# adsorption
p.epsilon = -1e-3
# protein mobility 
p.Bc = 1

g = dg.System(spineFace, spineVertex, p, o)

The additional parameters is `epsilon` and `Bc` that represents the unit binding energy $\epsilon$ and protein mobility compared to shape evolution. Since we do not model shape evolution, the speed can be set to 1 since the speed should reflected on the time step. The adsorption energy $E_a = \epsilon \int_{\mathcal{M}} \phi~ dA$ represents the base level membrane-protein interaction. Normally $\epsilon$ is negative to reflect the fact the protein tend to spontaneously bind to membrane.

In [7]:
fe = dg.Euler(f = g, dt = 30, total_time = 350000, tSave = 3000, tolerance = 1e-6, outputDir = "output/tutorial6/traj/")
fe.verbosity = 5
fe.integrate()

The resultant trajectory snapshots at `T = 0, 6000, 51000 and 300000' with the colormap showing the protein density: 

<img src="output/tutorial6/screenshot_frame000000.png" width="400" height="200">
<img src="output/tutorial6/screenshot_frame000002.png" width="400" height="200">
<img src="output/tutorial6/screenshot_frame000017.png" width="400" height="200">
<img src="output/tutorial6/screenshot_frame000100.png" width="400" height="200">
